In [ ]:
# Run for the first time to setup.
# pip install findspark

In [ ]:
# Run for the first time to setup.
# pip install pyspark

* These 2 statements are required to ensure that the notebook finds the installation of spark on our machine and initializes the required paths/variables for spark to work locally. Local spark testing works pretty well for smaller datasets when the laptop is pretty well configured. I have 32GB of RAM and I can test even medium sized datasets pretty easily.

In [ ]:
import findspark
findspark.init()

* This snippet shows us how to run the spark pi program inside the notebook via VSCODE and see the output. This is a good sanity testing exercise to ensure we do not have any environment issues.

In [ ]:
import pyspark
import random
sc = pyspark.SparkContext(appName="Pi")
num_samples = 100000000
def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1
count = sc.parallelize(range(0, num_samples)).filter(inside).count()
pi = 4 * count / num_samples
print(pi)
sc.stop()

* Now we can do some custom spark testing to ensure that we can do some computes passing some local datasets and getting somee basic aggregates.

In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import min
from pyspark.sql.functions import max

TEMPERATURES_DS_PATH = "file:///C:/Venky/AzureSynapseExperiments/datafiles/AirQualityIndexWithTemperatures_5/"
spark = SparkSession \
    .builder \
    .appName("Temperatures Analytics") \
    .getOrCreate()
    
print("Created spark session " + str(spark))

temperatures_ds = spark.read.parquet(TEMPERATURES_DS_PATH)
temperatures_ds.show(10)

print("Calculating min and max temperatures per lat lng")
temperatures_agg = temperatures_ds.groupBy("latitude", "longitude").agg(
    min(col("temperature_2m")).alias("min_temp"),
    max(col("temperature_2m")).alias("max_temp")
)

temperatures_agg.show()

Created spark session <pyspark.sql.session.SparkSession object at 0x0000024D25F29C50>
+---------+---------+----------------+--------------+
| latitude|longitude|            time|temperature_2m|
+---------+---------+----------------+--------------+
|44.699997|44.199997|2018-01-01T00:00|           0.9|
|44.699997|44.199997|2018-01-01T01:00|           0.9|
|44.699997|44.199997|2018-01-01T02:00|           0.8|
|44.699997|44.199997|2018-01-01T03:00|           0.8|
|44.699997|44.199997|2018-01-01T04:00|           0.9|
|44.699997|44.199997|2018-01-01T05:00|           1.3|
|44.699997|44.199997|2018-01-01T06:00|           1.9|
|44.699997|44.199997|2018-01-01T07:00|           2.7|
|44.699997|44.199997|2018-01-01T08:00|           3.3|
|44.699997|44.199997|2018-01-01T09:00|           4.1|
+---------+---------+----------------+--------------+
only showing top 10 rows

Calculating min and max temperatures per lat lng
+----------+-----------+-------------------+-------------------+
|  latitude|  long